# P-Median

Explicar aqui un poco el contexto de p-median y quizas pegar su formula

### 1. Bibliotecas

In [4]:
import json
import numpy as np

### 2. Configuración del proyecto

En la siguiente celda, se llama al archivo encontrado en la carpeta 'config', la cual contiene un archivo de configuración llamado 'config.json'. Este archivo declara los parámetros los cuales es posible modificar para ejecutar un experimento con diferentes características. Las características del archivo son:

```json
{
    "distance_data_path": "string", //Ruta del archivo de distancia.
    "demand_data_path": "string", //Ruta del archivo de demanda.
    "type_initial_solution": "string", //Posibles valores son: "random", ""
    "p": 0 //Cantidad de instalaciones a localizar
}
```


In [50]:
def readConfigFile(config_path=""):
    """
        Permite leer un archivo JSON.
        
        Parámetros:
            - config_path: ruta de archivo json a leer.
        
        Retorna:
            - data: dict con valores de archivo json.
    """
    assert config_path != "", "Config path is empty"
    
    with open(config_path) as f:
        data = json.load(f)
        
    return data

In [51]:
config_path = "./config/config.json"
config = readConfigFile(config_path)

### 3. Leer conjunto de datos

Se elaboran funciones para leer los archivos de distancia y demanda.

In [52]:
def readORLibDataFile(data_path = ""):
    if data_path == "":
        print("Empty data path, change it in config step.")
        return []
    
    data = []
    raw_data = open(data_path, "r")
    for line in raw_data:
        line = line.strip().split(" ")
        line = list(map(int, line))
        data.append(line)
        
    return data

def readTestDataFile(data_path = "", type_file=""):
    """
        Permite leer los archivos .txt encontrados en la carpeta 'data/test'. 
        Ambos archivos pertenecen a un ejemplo elaborado para probar el desarrollo de los algoritmos.
        
        Parámetros:
            - data_path: Ruta al archivo. Si este es vacío se entrega un mensaje de error.
            - type_file: String con valores posibles "distance" o "demand", permite ejecutar la lectura del archivo.
            
        Retorna:
            - data: Matriz con los datos pertinentes.
    """
    assert data_path != "", "data_path is empty."
    assert type_file != "", "type_file is empty."
    assert type_file in ["distance", "demand"], "type_file value unkown."
    
    if type_file == "distance":
        data = []
        raw_data = open(data_path, "r")
        for line in raw_data:
            line = line.strip().split(",")
            line = list(map(int, line))
            data.append(line)
        return data
    elif type_file == "demand":
        data = []
        raw_data = open(data_path, "r")
        for line in raw_data:
            line = line.strip().split(",")
            line = list(map(int, line))
            data = line
        return data
        
    else:
        print("Error; se desconoce el archivo que estás leyendo.")
        return []

    
def print_matrix(m):
    """
        Permite imprimir una matriz entregada como parámetro.
        
        Parámetros:
            - m: Matriz de nxn
        Retorna:
            
    """
    l = len(m)
    for i in range(l):
        s = ""
        for j in range(l):
            s += str(m[i][j]) + " "
        print(s)

In [249]:
distance = readTestDataFile(config["distance_data_path"], type_file = "distance")
demand = readTestDataFile(config["demand_data_path"], type_file = "demand")

#print("Distance: {}".format(type(distance)))
#print_matrix(distance)
#print("---------")
#print("Demand: {}".format(type(demand)))
#print(demand)

### 3. Generar solución inicial

Para ambas heurísticas utilizadas se necesita una solución inicial. A continuación, se declaran las definciones de las funciones que generar una solución inicial.

In [54]:
def generateRandomSolution(p, size):
    """
        Permite generar un vector de tamaño 'size', el cual contiene 'p' valores 1. Estos valores representan la posición en la cual se localiza el punto limpio.
        
        Parámetros:
            - p: 
            - size:
        Retorna:
            - x: 
    """
    x = np.zeros(size, dtype=int)
    x[:p] = 1
    np.random.shuffle(x)
    return x

### 5. Función objetivo

Se declara la función objetivo para el problema de 'p-median'.

In [250]:
def selectFacility(distance_row, permutation, distance):
    """
        Permite escoger una instalación con el menor valor de distancia dada una solución.
        
        Parámetros:
            - distance_row: vector de tamaño n, representa una fila en la matriz de distancia.
            - permutation: vector binario de tamaño n, representa una posible solucion del problema.
            - distance: matriz de enteros de tamaño (n,n), representa la distancia entre las localizaciones.
            
        Retorna:
            - min_index: entero, representa el indice de permutation en que se sirve una instalación con menor distancia.
    """
    #Determinados la mayor distancia de la fila
    n = len(permutation)
    min_value = max(distance_row)
    min_index = 0
    
    for i in range(n):
        if permutation[i] == 1:
            if distance_row[i] <= min_value:
                min_value = distance_row[i]
                min_index = i
    
    return min_index


def pMedianObjectiveFunction(permutation, distance, demand):
    """
        Permite calcular la función objetivo para 'p-median', en donde primero iteramos sobre las localizaciones,
        luego sobre cada fila de la matriz de distancia extraímos el indice de la menor distancia posible considerando la permutación o solucón dada.
        Se genera otra matriz binaria de instalaciones seleccionadas (selected_facilities), nos permite saber que localizaciones se sirven de otras.
        Finalmente total es valor de la función objetivo obtenida dada una permutación.
        
        Parámetros:
            - permutation: 
            - distance: 
            - demand:
        
        Retorna:
            - total:
            - selected_facilities:
        
    """
    n = len(demand)
    selected_facilities = []
    
    total = 0
    #Loop over locations
    for i in range(n):
        #Indice de donde se alimenta la planta
        min_index = selectFacility(distance[i], permutation, distance)
        
        #Creamos una fila y guardamos la distancia seleccionada
        aux = [0] * n
        aux[min_index] = 1
        
        #Lo agregamos a las instalaciones seleccionadas
        selected_facilities.append(aux)
        
        #Obtenemos el valor de la demanda * la distancia minima, lo agregamos al total
        total += demand[i] * distance[i][min_index]
        
    return total, selected_facilities

In [251]:
solution = [0,0,0,1,0,1]
total, sf = pMedianObjectiveFunction(solution, distance, demand)
print(total)
print_matrix(sf)

171
0 0 0 1 0 0 
0 0 0 1 0 0 
0 0 0 1 0 0 
0 0 0 1 0 0 
0 0 0 0 0 1 
0 0 0 0 0 1 


### 6. Heurísticas

Se desarrollan las siguientes herísticas para el problema de 'p-median':

- Algoritmo miópico.
- Heurística de Maranzana.
- Heurística de Teitz y Bart.

In [183]:
def myopic_algorithm(p = 0, distance = [], demand = []):
    """
        Fuente: https://mssanz.org.au/modsim2015/J11/dzator.pdf
        Algoritmo miópico elaborado por Daskin (1995), permite ir construyendo una solución integrando una instalación a localizar a la vez.
        Es un algoritmo de tipo 'greedy', por lo que es fácil que quede estancado en un mínimo local.
        
        Parámetros:
            - p: Entero, instalaciones a localizar.
            - distance: Matriz de (n,n) de enteros, representa la distancia entre las localizaciones.
            - demand: Vector de tamaño n de enteros, representa la demanda para la ubicacion.
            
        Retorna:
            - solution: Vector binario (1, 0) de tamaño n, representa donde están localizadas las p instalaciones.  
            - of_value: Entero, valor de la función objetivo p-median.
            - selected_facilities: Matriz binaria de (n,n) de enteros, representa la conexión entre las instalaciones localizadas.
    """
    assert p > 0, "p is equal or less than 0."
    assert p <= len(demand), "p must be the same dimension as demand."
    assert demand != [], "demand vector is empty."
    assert distance != [], "distance matrix is empty."
    
    k = 0
    m = len(demand)
    solution = np.zeros(m,dtype=int)
    
    #Colocamos 1 instalacion a la vez
    while k < p:
        #lowest_cost debe iniciarse en un valor muy alto
        lowest_cost = 99999
        lowest_cost_index = -1
        
        #Por cada ubicacion en la solucion
        temp = solution.copy()
        print(temp)
        for i in range(m):
            #Si esta ubicacion no ha sido seleccionada
            if temp[i] != 1:
                #La seleccionamos
                temp[i] = 1
                #Evaluamos la solucion
                print(temp)
                of_value, _ = pMedianObjectiveFunction(temp, distance, demand)
                #Si reduce la FO, Se selecciona como candidato
                if of_value <= lowest_cost:
                    lowest_cost = of_value
                    lowest_cost_index = i
                
                #Se retorna a 0 para seguir computando el resto de posibles valores
                temp[i] = 0
            
        #Si se encontró un candidato, se establece
        if lowest_cost_index != -1:
            solution[lowest_cost_index] = 1
            
        #Siguiente nodo p
        print("k: {}".format(k))
        #print_matrix(selected_facilities)
        print("-----")
        k += 1
    
    #Evaluamos finalmente nuestra solucion y entregamos los resultados.
    #print_matrix(selected_facilities)
    print(solution)
    of_value, selected_facilities = pMedianObjectiveFunction(solution, distance, demand)
    return solution, of_value, selected_facilities

In [234]:
myopic_algorithm(3, distance, demand)

[0 0 0 0 0 0]
[1 0 0 0 0 0]
[0 1 0 0 0 0]
[0 0 1 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 1 0]
[0 0 0 0 0 1]
k: 0
-----
[0 0 0 1 0 0]
[1 0 0 1 0 0]
[0 1 0 1 0 0]
[0 0 1 1 0 0]
[0 0 0 1 1 0]
[0 0 0 1 0 1]
k: 1
-----
[1 0 0 1 0 0]
[1 1 0 1 0 0]
[1 0 1 1 0 0]
[1 0 0 1 1 0]
[1 0 0 1 0 1]
k: 2
-----
[1 0 0 1 0 1]


(array([1, 0, 0, 1, 0, 1]),
 101,
 [[1, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0, 0],
  [0, 0, 0, 1, 0, 0],
  [0, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 1]])

In [153]:
def area_facility(n, index, selected_facilities, distance, demand):
    area = []
    
    temp = [j[index] for j in selected_facilities]
    for j in range(n):
        if temp[j] == 1:
            area.append(temp)
        else:
            area.append([0] * n)
        
    return area


def calculate_cost(area, distance, demand):
    n = len(area[0])
    temp = [0] * n
    for i in range(n):
        for j in range(n):
            temp[j] += area[i][j] * distance[i][j] * demand[i]
        
    
    return temp

def changeSelectedFacility(index, selected_facilities, new_cost):
    temp = [j[index] for j in selected_facilities]
    m = len(temp)
    
    lowest_cost_value = max(new_cost)
    lowest_cost_index = -1
    
    for i in range(m):
        if new_cost[i] != 0 and new_cost[i] < lowest_cost_value:
            lowest_cost_value = new_cost[i]
            lowest_cost_index = i
    
    return lowest_cost_value, lowest_cost_index

def createInitialSolution(t, p, size):
    if t == "random":
        return randomInitialSolution(p, size)
    else:
        print("not implemented yet")
        return []



def solveMaranzana(type_initial_solution, p, distance, demand):
    #Create initial solution
    
    #solution = createInitialSolution(type_initial_solution, p, len(demand))
    #Borrar, esto es por mientras
    solution = type_initial_solution
    
    #Calculate of_value of initial solution
    of_value, selected_facilities = pMedianObjectiveFunction(solution, distance, demand)
    
    #Chequear mejor solución para cada lugar seleccionado
    m = len(solution)
    is_changed = True

    while is_changed is True:
        solution_changed = False
        new_solution = solution.copy()
        for i in range(m):
            if solution[i] == 1:
                area = area_facility(m, i, selected_facilities, distance, demand)
                new_cost = calculate_cost(area, distance, demand)
                lowest_cost_value, lowest_cost_index = changeSelectedFacility(i, selected_facilities, new_cost)
                
                if i != lowest_cost_index and lowest_cost_index != -1:
                    new_solution[i] = 0
                    new_solution[lowest_cost_index] = 1
                    solution_changed = True
                
        if solution_changed:
            of_value, selected_facilities = pMedianObjectiveFunction(new_solution, distance, demand)
            solution = new_solution
        else:
            is_changed = False
    
    return solution, of_value, selected_facilities

In [230]:
solution = [0,0,0,0,1,1]         

sol, of_value, sf = solveMaranzana(solution, 2, distance, demand)
print(of_value)
print(sol)
print_matrix(sf)

161
[1, 0, 0, 1, 0, 0]
1 0 0 0 0 0 
0 0 0 1 0 0 
0 0 0 1 0 0 
0 0 0 1 0 0 
0 0 0 1 0 0 
0 0 0 1 0 0 
